In [107]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

In [108]:
data=pd.read_csv("data/an/anneal_train.csv")
# data.replace('?', np.nan).fillna(method='pad')

In [109]:
# feats = [f for f in data if f not in ['id','target']]
# for col in feats:
#     # 将每一列按照百分位数分成 5 个等级
#     data[col] = pd.qcut(data[col], q=4, labels=False, duplicates='drop')
# 输出离散化后的数据
data.isnull().sum()
# print(data)

id        0
f1        0
f2        0
f3        0
f4        0
f5        0
f6        0
f7        0
f8        0
f9        0
f10       0
f11       0
f12       0
f13       0
f14       0
f15       0
f16       0
f17       0
f18       0
f19       0
f20       0
f21       0
f22       0
f23       0
f24       0
f25       0
f26       0
f27       0
f28       0
f29       0
f30       0
f31       0
f32       0
f33       0
f34       0
f35       0
f36       0
f37       0
f38       0
target    0
dtype: int64

In [110]:
def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

In [111]:
import math
def fitness_function(condition_attrs, data, decision_attr):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    equivalence_classes = calculate_equivalence_classes(data, condition_attrs)
    print(len(equivalence_classes))
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)
    print(len(equivalence_classes_dec))
    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    print('-----------------')
    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        print('----------')
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            if(len(x)>0):
                total_rule+=len(x)
                print(len(x))
                print(class_size)
                y = 1-len(x) / (class_size+total_records)
                print(y)
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
        print('++++++++++++')

    rule_confidences = np.array(rule_confidences)
    print(rule_confidences)
    count = len(rule_confidences)
    print(count)
    # if count == 0:
    #     return 999;
    # if total_rule/total_records<0.4:
    #     return 999

    print(total_rule)
    avg_confidence = np.sum(rule_confidences) / count +0.1*(entropy_D-entropy_DB)

    return avg_confidence

In [113]:
# feature_columns = [col for col in data.columns if col not in ['id','target']]
feature_columns=['f19', 'f23', 'f26']
print(fitness_function(feature_columns, data, 'target'))

1
5
-----------------
----------
8
798
0.9949874686716792
88
798
0.9448621553884712
608
798
0.6190476190476191
60
798
0.9624060150375939
34
798
0.9786967418546366
++++++++++++
[0.99498747 0.94486216 0.61904762 0.96240602 0.97869674]
5
798
0.9


In [101]:
def generate_population(size, feature_columns):
    # 初始化种群
    population = []
    for i in range(size):
        individual = random.sample(feature_columns, random.randint(3, len(feature_columns) - 1))
        population.append(individual)
    return population

def select_parents(population, fitness_values, elite_size):
    elite_indices = np.argsort(fitness_values)[:elite_size]
    elite_population = [population[i] for i in elite_indices]
    population_without_elite = [individual for i, individual in enumerate(population) if i not in elite_indices]
    # 将相应的适应度函数值也从 fitness_values 中删除
    fitness_values_without_elite = [fitness_values[i] for i in range(len(fitness_values)) if i not in elite_indices]
    probabilities = (1 / (np.array(fitness_values_without_elite) + [1e-10])) / sum(1 / (np.array(fitness_values_without_elite) + [1e-10]))
    parents = []
    for _ in range(len(population) - elite_size):
        parent1_index = np.random.choice(len(population_without_elite), p=probabilities)
        parent2_index = np.random.choice(len(population_without_elite), p=probabilities)
        parents.append(population_without_elite[parent1_index])
        parents.append(population_without_elite[parent2_index])
    parents.extend(elite_population)
    return parents


def crossover(parents, crossover_prob):
    offspring = []
    if(len(parents)%2!=0):
        parents.pop()
    for i in range(0, len(parents)-1, 2):
        if random.random() < crossover_prob:
            # 随机选择交叉点，注意不能选择第一个或最后一个位置
            crossover_point = random.randint(1, len(parents[i]) - 2)
            # 生成两个新的后代
            child1 = parents[i][:crossover_point] + parents[i+1][crossover_point:]
            child2 = parents[i+1][:crossover_point] + parents[i][crossover_point:]
            offspring.extend([child1, child2])
        else:
            # 如果没有进行交叉操作，则直接将两个父代加入后代中
            offspring.extend([parents[i], parents[i+1]])
    return offspring

def mutate(individual, mutation_prob, feature_columns):
    mutated_individual = individual.copy()

    for i in range(len(mutated_individual) - 1):
        if random.random() < mutation_prob:
            attr = mutated_individual[i]
            other_attrs = [col for col in feature_columns if col != attr]
            if other_attrs:
                mutated_individual[i] = random.choice(other_attrs)
    return mutated_individual

In [102]:
def run_genetic_algorithm(data, decision_attr, population_size=50, max_generations=100, crossover_prob=0.8, mutation_prob=0.2, elite_size=5):
    feature_columns = [col for col in data.columns if col not in ['target','id']]
    # 初始化种群
    population = generate_population(population_size,feature_columns)
    # 计算每个个体的适应度函数值

    fitness_values = []
    for feature_columns in tqdm(population):
        fitness = fitness_function(feature_columns, data, decision_attr)
        fitness_values.append(fitness)
    # fitness_values = [fitness_function(feature_columns, data, decision_attr) for feature_columns in population]
    # 记录最优个体的适应度函数值和基因型
    best_fitness_value = min(fitness_values)
    best_individual = population[np.argmin(fitness_values)]
    # 进化过程
    for generation in tqdm(range(max_generations)):
        # 选择父母
        selected_parents = select_parents(population, fitness_values,elite_size)
        # 按一定概率进行交叉操作
        offspring = crossover(selected_parents, crossover_prob)
        # 按一定概率进行变异操作
        offspring = [mutate(individual, mutation_prob, feature_columns) for individual in offspring]
        # 计算每个个体的适应度函数值
        offspring_fitness_values = [fitness_function(individual, data, decision_attr) for individual in offspring]

        # 合并父代和子代
        combined_population = population + offspring
        combined_fitness_values = fitness_values + offspring_fitness_values

        # 按适应度函数值从小到大排序
        sorted_indices = np.argsort(combined_fitness_values)

        combined_population = [combined_population[i] for i in sorted_indices]
        combined_fitness_values = [combined_fitness_values[i] for i in sorted_indices]

        # 更新最优个体
        if combined_fitness_values[0] < best_fitness_value:
            best_fitness_value = combined_fitness_values[0]
            best_individual = combined_population[0]
        print("第轮最优个体")
        print(best_individual)
        print(best_fitness_value)
        # 选择新的种群
        population = combined_population[:population_size]
        fitness_values = [fitness_function(individual, data, decision_attr) for individual in population]
    return best_individual, best_fitness_value

In [103]:
import time
start_time = time.time()
print(run_genetic_algorithm(data, 'target', population_size=20, max_generations=40, crossover_prob=0.8,
                            mutation_prob=0.01, elite_size=1))
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

  0%|          | 0/20 [00:00<?, ?it/s]

      id  f1  f2  f3  f4  f5  f6  f7  f8  f9  ...  f30  f31  f32    f33  \
0      1   0   0   1   4   0   0   2   4   0  ...    0    0    0  0.700   
1      2   0   0   4   0   0   0   2   1   0  ...    0    0    0  3.200   
2      3   0   0   4   0   0   0   2   1   0  ...    0    0    1  0.700   
3      4   0   0   1   0   3   1   0   4   0  ...    0    0    0  2.801   
4      5   0   0   1   0   3   1   0   4   0  ...    0    0    1  0.801   
..   ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...  ...    ...   
793  794   0   0   1   0   2   1   0   4   0  ...    0    0    0  1.001   
794  795   0   0   0   0   0   0   2   0   0  ...    0    0    1  0.699   
795  796   0   0   1   0   0   0   2   1   0  ...    0    0    0  0.400   
796  797   0   0   1   0   0   0   2   1   0  ...    0    0    1  3.200   
797  798   0   0   4   0   0   0   2   1   0  ...    0    0    1  1.599   

        f34  f35  f36  f37  f38  target  
0     610.0    0    0    0    2       2  
1     610.0    

 10%|█         | 2/20 [00:00<00:03,  5.01it/s]


1
0.9987484355444305
++++++++++++
----------
2
2
0.9975
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
2
2
0.9975
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
----------
1
1
0.9987484355444305
++++++++++++
--

 15%|█▌        | 3/20 [00:00<00:04,  3.54it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

      id  f1  f2  f3  f4  f5  f6  f7  f8  f9  ...  f30  f31  f32    f33  \
0      1   0   0   1   4   0   0   2   4   0  ...    0    0    0  0.700   
1      2   0   0   4   0   0   0   2   1   0  ...    0    0    0  3.200   
2      3   0   0   4   0   0   0   2   1   0  ...    0    0    1  0.700   
3      4   0   0   1   0   3   1   0   4   0  ...    0    0    0  2.801   
4      5   0   0   1   0   3   1   0   4   0  ...    0    0    1  0.801   
..   ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...  ...    ...   
793  794   0   0   1   0   2   1   0   4   0  ...    0    0    0  1.001   
794  795   0   0   0   0   0   0   2   0   0  ...    0    0    1  0.699   
795  796   0   0   1   0   0   0   2   1   0  ...    0    0    0  0.400   
796  797   0   0   1   0   0   0   2   1   0  ...    0    0    1  3.200   
797  798   0   0   4   0   0   0   2   1   0  ...    0    0    1  1.599   

        f34  f35  f36  f37  f38  target  
0     610.0    0    0    0    2       2  
1     610.0    

  0%|          | 0/40 [00:02<?, ?it/s]

367
      id  f1  f2  f3  f4  f5  f6  f7  f8  f9  ...  f30  f31  f32    f33  \
0      1   0   0   1   4   0   0   2   4   0  ...    0    0    0  0.700   
1      2   0   0   4   0   0   0   2   1   0  ...    0    0    0  3.200   
2      3   0   0   4   0   0   0   2   1   0  ...    0    0    1  0.700   
3      4   0   0   1   0   3   1   0   4   0  ...    0    0    0  2.801   
4      5   0   0   1   0   3   1   0   4   0  ...    0    0    1  0.801   
..   ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...  ...    ...   
793  794   0   0   1   0   2   1   0   4   0  ...    0    0    0  1.001   
794  795   0   0   0   0   0   0   2   0   0  ...    0    0    1  0.699   
795  796   0   0   1   0   0   0   2   1   0  ...    0    0    0  0.400   
796  797   0   0   1   0   0   0   2   1   0  ...    0    0    1  3.200   
797  798   0   0   4   0   0   0   2   1   0  ...    0    0    1  1.599   

        f34  f35  f36  f37  f38  target  
0     610.0    0    0    0    2       2  
1     610.0

KeyboardInterrupt: 

In [ ]:
from sklearn.datasets import load_wine
import pandas as pd
import numpy as np

In [ ]:
# 读取红酒数据集
wine=load_wine()

wine_df = pd.concat([pd.DataFrame(wine.data), pd.DataFrame(wine.target)] ,axis=1)
wine_df.columns=list(wine.feature_names) + ['target']  # 将数据特征名称与数据对应
wine_df['target'] = wine_df['target'].map(dict(zip(range(3), wine.target_names)))  # 显示类别名称

n = len(wine_df)+1
nlist = range(1,n)
wine_df['id'] = nlist
d = wine_df.pop('id')
wine_df.insert(0,'id', d)

wine_df['target'] = wine_df['target'].map({'class_0': 0, 'class_1': 1,'class_2':2})
wine_df.to_csv('data/wine.csv',index=False)

In [ ]:
Y = wine_df["target"]
# feature_columns = [ col for col in wine_df.columns if col not in ['id','target']]
# feature_columns=['flavanoids', 'alcalinity_of_ash', 'proanthocyanins', 'color_intensity']0.948离散 5
feature_columns=['flavanoids', 'alcohol', 'color_intensity']   #'proanthocyanins' 0.948加上proanthocyanins 0.98 离散 6
feature_columns=['proline', 'flavanoids', 'alcohol', 'hue', 'malic_acid'] #0.966 离散 4
feature_columns=['hue', 'magnesium', 'alcohol', 'malic_acid', 'proline', 'flavanoids'] #离散 3 0.9661
feature_columns=['alcohol', 'malic_acid', 'ash', 'proanthocyanins', 'color_intensity', 'od280/od315_of_diluted_wines']
feature_columns=['alcohol', 'magnesium', 'flavanoids', 'nonflavanoid_phenols', 'color_intensity', 'hue', 'proline']
feature_columns=['ash', 'magnesium', 'flavanoids', 'color_intensity', 'od280/od315_of_diluted_wines', 'proline']
X =wine_df[feature_columns]

In [ ]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X,Y,test_size=0.33,random_state=1)

In [ ]:
from sklearn import svm
import math
from sklearn.preprocessing import StandardScaler

def SVM(trainX,trainY,testX,testY):
    scaler = StandardScaler()
    scaler.fit(trainX)
    trainX3 = scaler.transform(trainX)
    testX3 = scaler.transform(testX)
    clf = svm.SVC(C=10, gamma=0.8, max_iter=200)
    clf.fit(trainX3, trainY)
    test_result = clf.predict(testX3)
    precision = np.sum(test_result == testY)/testY.shape[0]
    return precision

In [ ]:
SVM(trainX,trainY,testX,testY)